#### 1. 保存计算两个变量和的模型。

In [43]:
import tensorflow as tf
import os
tf.reset_default_graph()
v1 = tf.Variable(tf.random_normal([1], stddev=1, seed=1),name="v1")
v2 = tf.Variable(tf.random_normal([1], stddev=1, seed=1),name="v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
#位于 tf.train.Saver()之后的变量将不会被存储!
saver = tf.train.Saver(max_to_keep=1)
print("tf.Session")
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run([v1,v2,result]))
    print("call saver.save")
    saver.save(sess, "./leo1/model.ckpt")
    saver.export_meta_graph("./leo-export_metagraph/model.ckpkt.meta.json",as_text=True)

tf.Session
[array([-0.8113182], dtype=float32), array([-0.8113182], dtype=float32), array([-1.6226364], dtype=float32)]
call saver.save


#### 2. 加载保存了两个变量和的模型。

In [39]:
with tf.Session() as sess:
    saver.restore(sess, "./leo1/model.ckpt")
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from ./leo1/model.ckpt
[-1.6226364]


In [46]:
#看后面“从ckpt查看tensor name”
import tensorflow as tf
reader = tf.train.NewCheckpointReader("./leo1/model.ckpt")
global_variables = reader.get_variable_to_shape_map()
for variable in global_variables:
    print(">",variable,global_variables[variable])


> v2 [1]
> v1 [1]


#### 从ckpt查看tensor name        
tf.Saver得到 model.ckpt.index和model.ckpt.data-**文件，其中保存了所有变量的取值。    
其中model.ckpt.data文件通过SSTable格式存储的，可以大致理解为一个(key, value)列表。    
TensorFlow提供了tf.train.NewCheckpointReader类来查看保存的变量信息。

In [45]:
import tensorflow as tf
 
reader = tf.train.NewCheckpointReader('./leo1/model.ckpt')
 
global_variables = reader.get_variable_to_shape_map()
for variable_name in global_variables:
    print("variable_name:", variable_name)
    print("global_varibles:", global_variables[variable_name])
    
print("Value for variable v1 is",reader.get_tensor("v1"))


variable_name: v2
global_varibles: [1]
variable_name: v1
global_varibles: [1]
Value for variable v1 is [-0.8113182]


#### 3. 直接加载持久化的图。因为之前没有导出v3，所以这里会报错。

In [42]:
#恢复tensorflow图，也就是读取神经网络的结构，从而无需再次构建网络
saver = tf.train.import_meta_graph("./leo1/model.ckpt.meta")
v3 = tf.Variable(tf.random_normal([1], stddev=1, seed=1))

with tf.Session() as sess:
    saver.restore(sess, "./leo1/model.ckpt")
    print("v1", sess.run(v1))
    print("v2", sess.run(v2))
 #   print("v3", sess.run(v3))

INFO:tensorflow:Restoring parameters from ./leo1/model.ckpt
v1 [-0.8113182]
v2 [-0.8113182]


#### 4. 变量重命名。

In [5]:
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "other-v2")
saver = tf.train.Saver({"v1": v1, "v2": v2})

##### 读取placeholder和最终的输出结果
```
graph = tf.get_default_graph()    
images_placeholder=graph.get_tensor_by_name('input_placeholder:0')
labels_placeholder=graph.get_tensor_by_name('result_placeholder:0')
logits=graph.get_tensor_by_name('softmax_linear/add:0')#最终输出结果的tensor
```
##### #恢复所有变量，包括权值
```
saver.restore(sess, tf.train.latest_checkpoint('D:/Work/python/GomokuPython-master/wuziqi/logs'))
feed_dict=fill_feed_dict(_currentGame,images_placeholder,labels_placeholder)
sessOutput=sess.run(logits, feed_dict=feed_dict)
```    
**以前版本的restore是需要指定哪个文件，而最新版本的restore需要指定目录，因为以前版本save结果只有一个文件，而现在有多个。如果碰到以下错误，
 Data loss: not an sstable (bad magic number): perhaps your file is in a different file
 请检查你的restore函数传入的是文件名还是路径名。**
 

##### 5. 导出meta到summary，用tensorboard查看

In [ ]:
import tensorflow as tf
from tensorflow.python.platform import gfile
 
graph = tf.get_default_graph()
graphdef = graph.as_graph_def()
_ = tf.train.import_meta_graph('meta的路径/xxxx.meta')
summary = tf.summary.FileWriter('summary的路径', graph)

#### 6. 导出到meta graph json格式

In [ ]:
import tensorflow as tf
 
saver = tf.train.import_meta_graph("Saved_model/model.ckpt.meta")
saver.export_meta_graph("Saved_model/model.ckpt.meta.json")

#### 7.ckpt转换为pb模型

In [ ]:
from tensorflow.python.tools import inspect_checkpoint as chkp
import tensorflow as tf

saver = tf.train.import_meta_graph("./ade20k/model.ckpt-27150.meta", clear_devices=True)

#【敲黑板！】这里就是填写输出节点名称
output_nodes = ["xxx"] 

with tf.Session(graph=tf.get_default_graph()) as sess:
    input_graph_def = sess.graph.as_graph_def()
    saver.restore(sess, "./ade20k/model.ckpt-27150")
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess,
                                                                    input_graph_def,
                                                                    output_nodes)
    with open("frozen_model.pb", "wb") as f:
        f.write(output_graph_def.SerializeToString())

#### new version ckpt to pb

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

checkpoint = "model.ckpt-xxx"
graph_file = "xxx.pb"


def return_ops(candidate):
    ops = []
    if isinstance(candidate, (list, tuple)):
        for x in candidate:
            ops += return_ops(x)
    else:
        ops.append(candidate.op)

    return ops


def dump_graph():
    with tf.Graph().as_default():
        inputs = setup_input(dtype=tf.float32,
                             shape=[None, 224, 224, 3],
                             name='graph_input')

        outputs = model_inference(inputs, 1000)

        model_info = gen_info(inputs, outputs)
        print(model_info)

        saver = tf.train.Saver(tf.global_variables())
        dest_node = return_ops(outputs)

        with tf.Session() as sess:
            saver.restore(sess, checkpoint)
            cur_graphdef = sess.graph.as_graph_def()
            output_graphdef = graph_util.convert_variables_to_constants(
                sess, cur_graphdef, [n.name for n in dest_node])

            with tf.gfile.GFile(graph_file, 'wb') as gf:
                gf.write(output_graphdef.SerializeToString())

            with open(graph_file + '.info', 'w') as info_f:
                info_f.write(model_info)


def setup_input(dtype, shape, name=None):
    p_node = tf.Placeholder(dtype=dtype, shape=shape, name=name)
    return p_node


def gen_info(inp, o):
    info_text = '[input tensor]: {0}\n[input shape]: {1}\n'.format(
        inp.name, inp.get_shape())
    print("outp", o)
    info_text += '[output tensor]: {0}\n[output shape]: {1}\n'.format(
        o.name, o.get_shape())

    return info_text


def model_inference(images, num_classes):
    with tf.variable_scope('xxx'):
        logits = tf.xxx
    return logits


if __name__ == "__main__":
    dump_graph()
    print('dump finish!')


In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf
import time

time.clock()

x = tf.placeholder(tf.float32 ,[None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x,W) + b)

# 为了计算交叉熵，我们需要添加一个新的占位符用于输入正确值。
y_ = tf.placeholder(tf.float32, [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
# 在此，我们要求TF使用梯度下降算法，并以0.01的学习速率最小化交叉熵。
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

#创建Saver节点，并设置自动保存最近n=1次模型
saver = tf.train.Saver(max_to_keep=1)
saver_max_acc = 0 
print("saver_max_acc:",saver_max_acc)
for i in range(100):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_:batch_ys})
    correct_prediction = tf.equal(tf.argmax(y,1), tf.arg_max(y_, 1))
    #准确率应该用测试数据进行评估
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    racc = sess.run(accuracy,feed_dict={x: mnist.test.images, y_:mnist.test.labels})
    print("racc:",racc,saver_max_acc)
    if (i+1)%10 == 0:
        print('{0:0>2d}:{1:.4f}'.format((i+1),accuracy.eval(session=sess, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))
    # 添加判断语句，选择保存精度最高的模型
    if racc > saver_max_acc:
        print("saver.save",racc,saver_max_acc)
        saver.save(sess,'leo/mnist.ckpt',global_step=i+1)
        saver_max_acc = racc
sess.close()
print(time.clock())

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
saver_max_acc: 0
racc: 0.292 0
saver.save 0.292 0
racc: 0.4474 0.292
saver.save 0.4474 0.292
racc: 0.5245 0.4474
saver.save 0.5245 0.4474
racc: 0.4374 0.5245
racc: 0.5696 0.5245
saver.save 0.5696 0.5245
racc: 0.6433 0.5696
saver.save 0.6433 0.5696
racc: 0.6661 0.6433
saver.save 0.6661 0.6433
racc: 0.6254 0.6661
racc: 0.6799 0.6661
saver.save 0.6799 0.6661
racc: 0.6836 0.6799
10:0.6836
saver.save 0.6836 0.6799
racc: 0.7343 0.6836
saver.save 0.7343 0.6836
racc: 0.6642 0.7343
racc: 0.717 0.7343
racc: 0.6161 0.7343
racc: 0.6812 0.7343
racc: 0.709 0.7343
racc: 0.76 0.7343
saver.save 0.76 0.7343
racc: 0.8001 0.76
saver.save 0.8001 0.76
racc: 0.7817 0.8001
racc: 0.7748 0.8001
20:0.7748
racc: 0.8196 0.8001
saver.save 0.8196 0.8001
racc: 0.8108 0.8196
racc: 0.8284 0.8196
saver.save 0.8284 0.8196
racc: 0.

#### 详解TensorFlow查看ckpt中变量的几种方法

查看TensorFlow中checkpoint内变量的几种方法：    

查看ckpt中变量的方法有三种：    
     
在有model的情况下，使用tf.train.Saver进行restore    
使用tf.train.NewCheckpointReader直接读取ckpt文件，这种方法不需要model。    
使用tools里的freeze_graph来读取ckpt    
注意：    

如果模型保存为.ckpt的文件，则使用该文件就可以查看.ckpt文件里的变量。ckpt路径为 model.ckpt    
如果模型保存为.ckpt-xxx-data (图结构)、.ckpt-xxx.index (参数名)、.ckpt-xxx-meta (参数值)文件，则需要同时拥有这三个文件才行。 并且ckpt的路径为 model.ckpt-xxx    
1. 基于model来读取ckpt文件里的变量    

首先建立model    
从ckpt中恢复变量    
```
with tf.Graph().as_default() as g:

  #建立model

  images, labels = cifar10.inputs(eval_data=eval_data)

  logits = cifar10.inference(images)

  top_k_op = tf.nn.in_top_k(logits, labels, 1)

  #从ckpt中恢复变量

  sess = tf.Session()

  saver = tf.train.Saver() #saver = tf.train.Saver(...variables...) # 恢复部分变量时，只需要在Saver里指定要恢复的变量

  save_path = 'ckpt的路径'

  saver.restore(sess, save_path) # 从ckpt中恢复变量
```
注意：基于model来读取ckpt中变量时，model和ckpt必须匹配。

1. 使用tf.train.NewCheckpointReader直接读取ckpt文件里的变量，使用tools.inspect_checkpoint里的print_tensors_in_checkpoint_file函数打印ckpt里的东西    
```
#使用NewCheckpointReader来读取ckpt里的变量    

from tensorflow.python import pywrap_tensorflow

checkpoint_path = os.path.join(model_dir, "model.ckpt")

reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path) #tf.train.NewCheckpointReader

var_to_shape_map = reader.get_variable_to_shape_map()

for key in var_to_shape_map:

  print("tensor_name: ", key)

  #print(reader.get_tensor(key))
#使用print_tensors_in_checkpoint_file打印ckpt里的内容

from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file


print_tensors_in_checkpoint_file(file_name, #ckpt文件名字

                 tensor_name, # 如果为None,则默认为ckpt里的所有变量

                 all_tensors, # bool 是否打印所有的tensor，这里打印出的是tensor的值，一般不推荐这里设置为False

                 all_tensor_names) # bool 是否打印所有的tensor的name
```               

#上面的打印ckpt的内部使用的是pywrap_tensorflow.NewCheckpointReader所以，掌握NewCheckpointReader才是王道

1. 使用tools里的freeze_graph来读取ckpt
```
from tensorflow.python.tools import freeze_graph


freeze_graph(input_graph, #=some_graph_def.pb

       input_saver,

       input_binary,

       input_checkpoint, #=model.ckpt

       output_node_names, #=softmax

       restore_op_name,

       filename_tensor_name,

       output_graph, #='./tmp/frozen_graph.pb'

       clear_devices,

       initializer_nodes,

       variable_names_whitelist='',

       variable_names_blacklist='',

       input_meta_graph=None,

       input_saved_model_dir=None,

       saved_model_tags='serve',

       checkpoint_version=2)

#freeze_graph_test.py讲述了怎么使用freeze_grapg。
使用freeze_graph可以将图和ckpt进行合并
```